In [32]:
import sys
import os
parent = os.path.abspath("..")
sys.path.append(parent)
configpath = os.path.join(parent, "test.ini")
import numpy as np
import importlib
from configobj import ConfigObj
from scipy.linalg.lapack import dgesv
from TackleBox import Set_Bait, Fish, CovRenorm
from ioutils import CosmoResults, InputData, write_fisher

# Read in the config file
pardict = ConfigObj(configpath)

# Read in the file containing the redshift bins, nz and bias values
data = InputData(pardict)

# Set up the linear power spectrum and derived parameters based on the input cosmology
cosmo = CosmoResults(pardict, data.zmin, data.zmax)
if np.any(data.volume > 0):
    cosmo.volume = data.volume * 1.0e9

# Convert the nz to nbar in (h/Mpc)^3
data.convert_nbar(cosmo.volume, float(pardict["skyarea"]))

# Scales the bias so that it goes as b/G(z)
if pardict.as_bool("scale_bias"):
    data.scale_bias(cosmo.growth)
print("#  Data nbar")
print(data.nbar)
print("#  Data bias")
print(data.bias)

# Precompute some things we might need for the Fisher matrix
recon, derPalpha, derPalpha_BAO_only = Set_Bait(cosmo, data, BAO_only=pardict.as_bool("BAO_only"))
print("#  Data recon factor")
print(recon)

# Loop over redshifts and compute the Fisher matrix and output the 3x3 matrix
identity = np.eye(len(data.nbar) + 3)
print(
    "#  z  V(Gpc/h)^3  fsigma8  fsigma8_err(%)  Da(Mpc/h)  Da_err(%)  H(km/s/Mpc)  H_err(%)   alpha_err(%)"
)
erralpha = np.zeros(len(cosmo.z))
FullCatch = np.zeros((len(cosmo.z) * len(data.nbar) + 3, len(cosmo.z) * len(data.nbar) + 3))
for iz in range(len(cosmo.z)):

    if np.any(data.nz[:, iz] > 1.0e-30):

        Catch = Fish(
            cosmo,
            cosmo.kmin,
            cosmo.kmax,
            data,
            iz,
            recon[iz],
            derPalpha,
            pardict.as_bool("BAO_only"),
            pardict.as_bool("GoFast"),
        )
        # print(Catch)

        # Add on BAO only information from kmax to k = 0.5 Mpc/h but only for alpha_perp and alpha_par
        ExtraCatch = Fish(
            cosmo,
            cosmo.kmax,
            0.5,
            data,
            iz,
            recon[iz],
            derPalpha_BAO_only,
            True,
            pardict.as_bool("GoFast"),
        )
        Catch[-2:, -2:] += ExtraCatch[-2:, -2:]

        # Add the Fisher matrix to the full fisher matrix
        FullCatch[
            iz * len(data.nbar) : (iz + 1) * len(data.nbar),
            iz * len(data.nbar) : (iz + 1) * len(data.nbar),
        ] += Catch[: len(data.nbar), : len(data.nbar)]
        FullCatch[iz * len(data.nbar) : (iz + 1) * len(data.nbar), -3:,] += Catch[: len(data.nbar), -3:]
        FullCatch[-3:, iz * len(data.nbar) : (iz + 1) * len(data.nbar)] += Catch[-3:, : len(data.nbar)]
        FullCatch[-3:, -3:] += Catch[-3:, -3:]

        # Invert the Fisher matrix to get the parameter covariance matrix
        cov = dgesv(Catch, identity)[2]

        # Compute the error on isotropic alpha also
        J = np.array([2.0 / 3.0, 1.0 / 3.0])
        erralpha[iz] = 100.0 * np.sqrt(J @ cov[-2:, -2:] @ J.T)

        # Renormalise the covariance from fsigma8, alpha_perp, alpha_par to fsigma8, Da, H
        means = np.array([cosmo.f[iz] * cosmo.sigma8[iz], cosmo.da[iz], cosmo.h[iz]])
        cov_renorm = CovRenorm(cov, means)

        # Print the parameter means and errors
        errs = 100.0 * np.sqrt(np.diag(cov_renorm)[-3:]) / means
        print(
            " {0:.2f}    {1:.4f}     {2:.3f}       {3:.2f}         {4:.1f}       {5:.2f}        {6:.1f}       {7:.2f}       {8:.3f}".format(
                cosmo.z[iz],
                cosmo.volume[iz] / 1e9,
                means[0],
                errs[0],
                means[1],
                errs[1],
                means[2],
                errs[2],
                erralpha[iz],
            )
        )

        # Output the fisher matrix for the redshift bin
        write_fisher(pardict, cov_renorm, cosmo.z[iz], means)

    else:
        erralpha[iz] = 1.0e30
        print(
            " {0:.2f}     {1:.4f}    {2:.3f}         -          {4:.1f}         -         {6:.1f}         -          -".format(
                cosmo.z[iz],
                cosmo.volume[iz] / 1e9,
                means[0],
                errs[0],
                means[1],
                errs[1],
                means[2],
                errs[2],
                erralpha[iz],
            )
        )

# Run the cosmological parameters at the centre of the combined redshift bin
identity = np.eye(len(cosmo.z) * len(data.nbar) + 3)
cosmo = CosmoResults(pardict, np.atleast_1d(data.zmin[0]), np.atleast_1d(data.zmax[-1]))

# Invert the Combined Fisher matrix to get the parameter
# covariance matrix and compute means and errors
cov = dgesv(FullCatch, identity)[2]
J = np.array([2.0 / 3.0, 1.0 / 3.0])
erralpha = 100.0 * np.sqrt(J @ cov[-2:, -2:] @ J.T)
means = np.array([cosmo.f[0] * cosmo.sigma8[0], cosmo.da[0], cosmo.h[0]])
cov_renorm = CovRenorm(cov, means)
errs = 100.0 * np.sqrt(np.diag(cov_renorm)[-3:]) / means
print("#  Combined errors")
print("#=================")
print(
    " {0:.2f}    {1:.4f}     {2:.3f}       {3:.2f}         {4:.1f}       {5:.2f}        {6:.1f}       {7:.2f}       {8:.3f}".format(
        cosmo.z[0],
        cosmo.volume[0] / 1e9,
        means[0],
        errs[0],
        means[1],
        errs[1],
        means[2],
        errs[2],
        erralpha,
    )
)

/root/enidh/code/capstone/Capstone-Project/ioutils.py:34: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  """


#  Data nbar
[[0.04580784]]
#  Data bias
[[1.37583381]]
#  Data recon factor
[0.5]
#  z  V(Gpc/h)^3  fsigma8  fsigma8_err(%)  Da(Mpc/h)  Da_err(%)  H(km/s/Mpc)  H_err(%)   alpha_err(%)
 0.05    0.0356     0.441       9.89         209.4       6.62        69.0       13.84       4.917
#  Combined errors
#=================
 0.05    0.0356     0.441       9.89         209.4       6.62        69.0       13.84       4.917
